https://plotly.com/python/v3/figurewidget-app/#version-check

Interactive Data Analysis with FigureWidget ipywidgets in Python/v3

**New to Plotly?**
Plotly's Python library is free and open source! Get started by downloading the client and reading the primer.
You can set up Plotly to work in online or offline mode, or in jupyter notebooks.
We also have a quick-reference cheatsheet (new!) to help you get started!

**Version Check**
Plotly's python package is updated frequently. Run pip install plotly --upgrade to use the latest version.

In [2]:
import plotly
plotly.__version__

'5.21.0'

**NYC Flights Database**

In [3]:
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets

In [4]:
from google.colab import output
output.enable_custom_widget_manager()

ModuleNotFoundError: No module named 'google.colab'

Support for third party widgets will remain active for the duration of the session. To disable support:

We'll be making an application to take a look at delays from all flights out of NYC in the year 2013.

In [5]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
df = df.drop(df.columns[[0]], axis=1)

In [6]:
df.sample(3)

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
90183,2013,12,8,1524.0,30.0,2011.0,20.0,DL,N3762Y,2003,JFK,SJU,196.0,1598,15.0,24.0
151995,2013,3,18,30.0,32.0,421.0,43.0,B6,N589JB,707,JFK,SJU,190.0,1598,0.0,30.0
300410,2013,8,22,1222.0,22.0,1422.0,27.0,US,N194UW,1979,JFK,CLT,86.0,541,12.0,22.0


Let's get the set of all the airlines, so that we can type the right things into the search box later.

In [7]:
df['carrier'].unique()

array(['UA', 'AA', 'B6', 'DL', 'EV', 'MQ', 'US', 'WN', 'VX', 'FL', 'AS',
       '9E', 'F9', 'HA', 'YV', 'OO'], dtype=object)

Let's assign the widgets that we're going to be using in our app. In general all these widgets will be used to filter the data set, and thus what we visualize.

1. Design

In [8]:
month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

myw1 = widgets.ColorPicker()

container = widgets.HBox(children=[use_date, month, myw1, widgets.DatePicker()])

textbox = widgets.Dropdown(
    description='Airline:   ',
    value='DL',
    options=df['carrier'].unique().tolist()
)

origin = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)



# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='NYC FlightDatabase'
                        ),
                        barmode='overlay'
                    ))

2. Action

Let now write a function that will handle the input from the widgets, and alter the state of the graph.

In [9]:
def validate():
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j and k for i, j, k in
                           zip(df['month'] == month.value, df['carrier'] == textbox.value,
                               df['origin'] == origin.value)]
            print(filter_list)
            temp_df = df[filter_list]

        else:
            filter_list = [i and j for i, j in
                           zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
            temp_df = df[filter_list]
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Delay in Minutes'
            g.layout.yaxis.title = 'Number of Delays'
            g.layout.plot_bgcolor = myw1.value


origin.observe(response, names="value")
textbox.observe(response, names="value")
month.observe(response, names="value")
use_date.observe(response, names="value")
myw1.observe(response,names="value")

Time to try the app out!!

In [10]:
container2 = widgets.HBox([origin, textbox])
widgets.VBox([container,
              container2,
              g])